In [6]:
import json
import os
import sys
sys.path.insert(0, '/home/CORP/fady.rezk/frezk/LaMP')

import torch
import numpy as np
from metrics.utils import get_metrics
from transformers import AutoTokenizer

max_generation_length = 512
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

/home/CORP/fady.rezk/miniconda3/envs/lamp/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
prefix = ''

# LaMP-2
lamp2 = {
    'SGD': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-2/sgd_baseline_new_split/r_6_alpha_16_lr_0.01_epochs_20_sch_linear/per_user', 'query_'
    ],
    'LoRA-Hub': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-2/dev_questions_merged_lora_hub_new_split/per_user', ''
    ],
    'MeZO': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-2/mezo_new_split/r_6_alpha_16_lr_0.001_epochs_320_sch_linear/per_user', 'query_'
    ],
    'Model Zoo': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-2/dev_questions_merged_stage_4_zoo_5_clusters_new_split/per_user/', ''
    ],
    'STEEL': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-2/dev_questions_merged_stage_4_mlp4x3l_22_clusters_zoo_new_split/per_user', ''
    ],
}

# LaMP-5
lamp5 = {
    'SGD': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-5/sgd_baseline/r_6_alpha_16_lr_0.01_epochs_20_sch_linear/per_user', 'query_'
    ],
    'SGD-new-recipe': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-5/sgd_baseline_new_recipe/r_6_alpha_16_lr_0.001_epochs_10_sch_linear/per_user', 'query_'
    ],
    # LoRA-Hub was truncated properly to 256 (checked in hyperparameters.json dump)
    'LoRA-Hub': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-5/dev_questions_merged_lora_hub/per_user', ''
    ],
    # LoRA-Hub was truncated properly to 256 (checked in hyperparameters.json dump)
    'MeZO': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-5/mezo/r_6_alpha_16_lr_0.001_epochs_320_sch_linear/per_user', 'query_'
    ],
    'Model Zoo': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-5/dev_questions_merged_stage_4_model_zoo_54clusters_trnct_256/per_user', ''
    ],
    'STEEL': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-5/dev_questions_merged_stage_4_mlp4x3l_14clusters_trnct_256/per_user', ''
    ],
}

# LaMP-3
lamp3 = {
    'SGD': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-3/sgd_baseline/r_6_alpha_16_lr_0.01_epochs_20_sch_linear/per_user', 'query_'
    ],
    'SGD-new-recipe': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-3/sgd_baseline_new_recipe/r_6_alpha_16_lr_0.01_epochs_10_sch_linear/per_user', 'query_'
    ],
    # LoRA-Hub was truncated properly to 256 (checked in hyperparameters.json dump)
    'LoRA-Hub': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-3/dev_questions_merged_lora_hub_trnct_256_seeded/per_user/', ''
    ],
    'Mezo': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-3/mezo/r_6_alpha_16_lr_0.001_epochs_320_sch_linear/per_user', 'query_'
    ],
    'Model Zoo': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-3/dev_questions_merged_stage_4_model_zoo_150clusters_trnct_256/per_user', ''
    ],
    'STEEL': [
        '/scratch-01/frezk/LaMP/experiments/LaMP-3/dev_questions_merged_stage_4_mlp4x3l_92clusters_trnct_256/per_user', ''
    ]
}


def tokenize(data, tokenizer):
    tokenized = tokenizer(data)['input_ids']
    tokenized = np.array([np.pad(torch.tensor(x), (tokenizer.pad_token_id, max_generation_length - len(x))) for x in tokenized])
    return tokenized

def flatten(data):
    if type(data[0]) is list:
        data = [x for data_list in data for x in data_list]
    return data

def get_labels_predictions(dir, prefix):
    users_files = [x for x in os.listdir(dir)]
    users = [os.path.join(dir, user) for user in users_files]
    print('found total', len(users), 'in dir')

    predictions = []
    labels = []
    for user in users:
        with open(user) as f:
            data = json.load(f)
        predictions.append(data[f'{prefix}pred'])
        labels.append(data[f'{prefix}label'])

    predictions = flatten(predictions)
    labels = flatten(labels)
    return labels, predictions

def get_results(labels, predictions, compute_metrics):
    tokenized_labels = tokenize(labels, tokenizer)
    tokenized_predictions = tokenize(predictions, tokenizer)
    results = compute_metrics((tokenized_predictions, tokenized_labels))
    return results

In [8]:
# LaMP-2
compute_metrics, best_metric, true_labels, greater_is_better = get_metrics('LaMP-2', tokenizer)
for method_name, properties in lamp2.items():
    dir, prefix = properties
    labels, predictions = get_labels_predictions(dir, prefix)
    print('#'*10, method_name)
    print(len(labels))
    print(get_results(labels, predictions, compute_metrics))

found total 691 in dir
########## SGD
5505
{'accuracy': 0.6325158946412353, 'f1': 0.5615734865842781}
found total 691 in dir
########## LoRA-Hub
5505
{'accuracy': 0.5751135331516803, 'f1': 0.5084813713561379}
found total 691 in dir
########## MeZO
5505
{'accuracy': 0.6330608537693007, 'f1': 0.5703730719383875}
found total 691 in dir
########## Model Zoo
5505
{'accuracy': 0.6252497729336967, 'f1': 0.552822772705473}
found total 691 in dir
########## STEEL
5505
{'accuracy': 0.6374205267938238, 'f1': 0.5569210340038492}


In [9]:
# get all subdirectories in the directory
compute_metrics, best_metric, true_labels, greater_is_better = get_metrics('LaMP-3', tokenizer)
for method_name, properties in lamp3.items():
    dir, prefix = properties
    labels, predictions = get_labels_predictions(dir, prefix)
    print('#'*10, method_name)
    print(len(labels))
    print(get_results(labels, predictions, compute_metrics))

found total 2500 in dir
########## SGD
2500


/home/CORP/fady.rezk/miniconda3/envs/lamp/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'mae': 0.2216, 'rmse': 0.52}
found total 2500 in dir
########## SGD-new-recipe
2500


/home/CORP/fady.rezk/miniconda3/envs/lamp/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'mae': 0.2176, 'rmse': 0.5114684741017769}
found total 2500 in dir
########## LoRA-Hub
2500


...

/home/CORP/fady.rezk/miniconda3/envs/lamp/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'mae': 0.2428, 'rmse': 0.5317894320123332}
found total 2500 in dir
########## Model Zoo
2500


/home/CORP/fady.rezk/miniconda3/envs/lamp/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'mae': 0.232, 'rmse': 0.5238320341483518}
found total 2500 in dir
########## STEEL
2500
{'mae': 0.2316, 'rmse': 0.5242136968832463}


/home/CORP/fady.rezk/miniconda3/envs/lamp/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [10]:
# get all subdirectories in the directory
compute_metrics, best_metric, true_labels, greater_is_better = get_metrics('LaMP-5', tokenizer)
for method_name, properties in lamp5.items():
    dir, prefix = properties
    labels, predictions = get_labels_predictions(dir, prefix)
    print('#'*10, method_name)
    print(len(labels))
    print(get_results(labels, predictions, compute_metrics))

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/CORP/fady.rezk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/CORP/fady.rezk/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/CORP/fady.rezk/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


found total 2500 in dir
########## SGD
2500
{'bleu': 12.12872426962075, 'rouge-1': 0.4647709784202234, 'rouge-2': 0.2752048488145653, 'rouge-L': 0.42223627975021216, 'rouge-LSum': 0.4222482549617859, 'meteor': 0.4272147029155192}
found total 2500 in dir
########## SGD-new-recipe
2500
{'bleu': 12.844655363513676, 'rouge-1': 0.47014985421219424, 'rouge-2': 0.27939695264388653, 'rouge-L': 0.42798701375986703, 'rouge-LSum': 0.42794780253637965, 'meteor': 0.43779202474032125}
found total 2500 in dir
########## LoRA-Hub
2500
{'bleu': 12.517179212810316, 'rouge-1': 0.4709681642200614, 'rouge-2': 0.2790096909961383, 'rouge-L': 0.4276295808916075, 'rouge-LSum': 0.4276532917128513, 'meteor': 0.4395102997971276}
found total 2500 in dir
########## MeZO
2500
{'bleu': 12.689938924069233, 'rouge-1': 0.4702196669272689, 'rouge-2': 0.27828620236614554, 'rouge-L': 0.4274714684273735, 'rouge-LSum': 0.4275178632982731, 'meteor': 0.4345054523548217}
found total 2500 in dir
########## Model Zoo
2500
{'bleu'